In [ ]:
!pip install yt_dlp


from traceback import print_exc
import urllib.request
# yt-dlp: https://github.com/yt-dlp/yt-dlp
import yt_dlp as ytdl
# Google Spread Sheets: https://docs.gspread.org/en/latest/api.html
import gspread
# Authentication: https://hituji-ws.com/code/python/python-spreadsheet
from oauth2client.service_account import ServiceAccountCredentials
# Authentication: for every time
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

class GSpread:
  def __init__(self, file_name, sheet_name):
    self.file_name = file_name
    self.sheet_name = sheet_name

  def sheet_(self):
    return gc.open(self.file_name).worksheet(self.sheet_name)
  
  def get_(self):
    data = self.sheet_().get_all_values()
    del data[0:2]
    dn = [ {'title': data[i][0], 'id': data[i][1]} for i in range(len(data)) ]
    return dn

  def append_(self, dl:list):
    dl = [ [val['title'], val['id']] for val in dl ]
    self.sheet_().append_rows(dl, table_range='A3')

def ytdl_info(url:str):
  opts = {
      'quiet': True,
      'sleep_interval': 1,
      'cookiefile': '/content/drive/MyDrive/Colaboratory/yt-dlp/youtube_cookies.txt',
      'geo_bypass': True,
      'ignoreerrors': True
  }
  try:
    dl = ytdl.YoutubeDL(opts)
    info = dl.extract_info(url, download=False)
    dl = [ {'title': val['title'], 'id': val['id']} for val in info['entries'] ]
  except KeyError as e:
    print(f'Couldn\'t find a value of {e} >>> Shift to ListMode')
    dl = [ {'title': info['title'], 'id': info['id']} ]
  return dl

def ytdl_dl(id:list):
  opts = {
      'outtmpl': '/content/drive/MyDrive/Music/Youtube/%(title)s.%(ext)s',
      'format': '251/bestaudio',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'opus',
          'preferredquality':'192'
      }],
      'cookiefile': '/content/drive/MyDrive/Colaboratory/yt-dlp/youtube_cookies.txt',
      'geo_bypass': True,
      'ignoreerrors': True
  }
  dl = ytdl.YoutubeDL(opts)
  for val in id:
    dl.extract_info(val, download=True)

def input_url():
  url = input('URL: ')
  if not check_url(url):
    return input_url()
  return url

def check_url(url:str):
  try:
    f = urllib.request.urlopen(url)
    f.close()
  except urllib.request.HTTPError:
    print('Not found: ', url)
    return False
  return True

if __name__ == "__main__":
  try:
    # Authentication: for every time
    # auth.authenticate_user()
    # gc = gspread.authorize(GoogleCredentials.get_application_default())
    json_file = '/content/drive/MyDrive/Colaboratory/yt-dlp/yt-dlp-colab-key.json'
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file, scope)
    gc = gspread.authorize(credentials)
    url = input_url()
    print('\n###Getting Title & ID of Done from spread sheet...###')
    gspread = GSpread('youtube', 'music')
    dn = gspread.get_()
    print('\n###Processing yt-dlp to get infomation...###')
    dl = ytdl_info(url)
    print('\n###Processing yt-dlp to download files...###')
    id = set([val['id'] for val in dl]) - set([val['id'] for val in dn])
    if not id:
      print('\n###Nothing to download!!###')
    else:
      ytdl_dl(id)
      print('\n###Updating a spread sheet...###')
      dl = [ {'title': val['title'], 'id': val['id']} for val in dl if val['id'] in id ]
      gspread.append_(dl)
      print('\n###Complete!!###')

  except Exception:
    print_exc()